In [19]:
import pandas as pd 
import numpy as np

import cleaning_functions as cf
import legal_functions as lf
import encoding_functions as ef
import bail_status_dict as bs

In [20]:
df1 = pd.read_csv("../data/data_1.csv")
print(df1.shape)
df2 = pd.read_csv("../data/data_2.csv")
print(df2.shape)
df3 = pd.read_csv("../data/data_3.csv")
print(df3.shape)
df4 = pd.read_csv("../data/data_4.csv")
print(df4.shape)

(843, 11)
(558, 11)
(980, 11)
(823, 11)


In [21]:
df1 = cf.split_info(df1)
df2 = cf.split_info(df2)
df3 = cf.split_info(df3)
df4 = cf.split_info(df4)

In [22]:
# concatenate vertically
dfA = pd.concat([df1, df2, df3, df4], axis=0)

# reset the index
dfA = dfA.reset_index(drop=True)

dfA.shape

(3204, 15)

### Import Offences Lookup Table, encode it, and see how many offences are covered. Drop those that aren't

In [23]:
lookup_df = pd.read_csv("../data/OffenceLookup.csv")
lookup_df.head()

,Statute,Offence Number,Offence,Election,Discharge Available,SS Available,CSO Available,Maximum (Summary)(Years),Maximum (Indictable)(Years)
0,CCC,56.1 1,Possess ID docs,Hybrid,Y,Y,Y,2,5
1,CCC,72 1,Forcible entry,Hybrid,Y,Y,Y,2,2
2,CCC,82 1,Make/possess explosives,Hybrid,Y,Y,Y,2,5
3,CCC,85 1,Use firearm in commission of offence,Indictable,N,N,N,5,14
4,CCC,85 2,Use imitation firearm in commission of offence,Indictable,N,N,N,5,14


In [24]:
lookup_df = lf.encode_offences(lookup_df)

In [25]:
# drop rows with offence codes that don't meet the criteria
dfA = cf.clean_offence_codes(dfA)

61 rows have been dropped


In [26]:
# compare df with lookup_df and drop rows that don't have an offence in the lookup table
dfA = lf.compare_dataframes(dfA, lookup_df)

Number of values in both dataframes (to keep): 3060
Number of values not in lookup df (to drop): 83
Offences that have no lookup and are being dropped: ['502 2' '771 1' '117.05 1' '11 2' '159 1' '512.3' 'CCC 811' '503 3 b'
 '117.05 4 b' '78' '111 1' '000' 'CCC 733.1 1' '486.2 2 T' '89.6 1'
 '37 1 b' '26 1 c' '254.1 1' '162 1 b' '732.2 3 a' '153' '445 1 a' '9.1 1'
 '320.13 3' '171.1 1 b' '486.1 2' '92 1' '209' '364 1' '507 1 a' '507.1 1'
 '430 4 a' '490 9.1' '430 1' '154 4 a' '279.01 1' '127 a']


Merge df1 with lookup_df

In [27]:
merged_df = dfA.merge(lookup_df, on = ['Offence Number'], how='left')

In [28]:
merged_df.head()

,File,Name,Release Type,In Custody,Lawyer,Days in Court,Info,Videoconf,Offence Location,Court Location,...,Statute_x,Offence Number,Statute_y,Offence,Election,Discharge Available,SS Available,CSO Available,Maximum (Summary)(Years),Maximum (Indictable)(Years)
0,93322-9-B,"Hampton, Braiden",AOI,Y,"Reveley,",100.0,JIR 001 CCC 811,NaN,Mission BC,Abbotsford,...,CCC,811,CCC,Breach of Recognizance,1,0,0,0,2,2
1,95217-2-B,"Hampton, Braiden Gray",AOI,N,"Reveley,",145.0,ARB 001 CCC 733.1 1,NaN,Abbotsford BC,Abbotsford,...,CCC,733.1 1,CCC,Breach of Probation,1,0,0,0,2,2
2,95824-1,"Hampton, Braiden Gray",AOI,N,"Reveley,",130.0,ARB 001 CCC 266,NaN,Abbotsford BC,Abbotsford,...,CCC,266,CCC,Assault,1,0,0,0,2,5
3,95935-2-C,"Hampton, Braiden Gray",PPA,Y,"Reveley,",104.0,JIR 001 CCC 266,NaN,Mission BC,Abbotsford,...,CCC,266,CCC,Assault,1,0,0,0,2,5
4,94351-1,"Harris, Ryan Patrick",ROD,Y,"Reveley,",391.0,JIR 001 CDS 5 2,NaN,Mission BC,Abbotsford,...,CDS,5 2,CDS,PPT,1,0,0,0,2,25


In [29]:
# export pre-encoded df for EDA
merged_df.to_csv("../data/pre-encoded_df.csv")

In [30]:
merged_df.columns

Index(['File', 'Name', 'Release Type', 'In Custody', 'Lawyer', 'Days in Court',
       'Info', 'Videoconf', 'Offence Location', 'Court Location', 'Date',
       'Proceeding Type', 'Court', 'Statute_x', 'Offence Number', 'Statute_y',
       'Offence', 'Election', 'Discharge Available', 'SS Available',
       'CSO Available', 'Maximum (Summary)(Years)',
       'Maximum (Indictable)(Years)'],
      dtype='object')

In [37]:
merged_df['Offence Number'].nunique()

159

In [31]:
merged_df['Offence'].nunique()

112

### Imputing and Encoding

In [32]:
# impute missing values for number of days in court
merged_df = cf.impute_days_in_court(merged_df)

In [33]:
# encode everything for modeling

merged_df = cf.clean_lawyers(merged_df)
merged_df = bs.map_release_type(merged_df)
merged_df = ef.encode_courts(merged_df)
merged_df = cf.drop_cols(merged_df)

columns: 'Videoconf', 'Info', 'File', 'Name', 'Lawyer', and 'Statute' have been dropped


c:\Users\emily\OneDrive\Desktop\Data Science Bootcamp\Final Project\src\cleaning_functions.py:66: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Lawyer'] = df['Lawyer'].str.replace(r'[^\w\s]','').str.replace(r'\b\w{1,2}\b','')


In [34]:
# there are 220 unique values in the 'offence' category. Experiment with one-hot v binary encoding
#(some loss of data for binary)

merged_df, very_large_df = ef.encode_offences(merged_df)

New dataframe created with 31 columns
New dataframe created with 136 columns


In [35]:
# the choice of whether to call encode_incustody or clean_incustody will depend on the 
# target (lawyer or bail) and can be done in the next step)

In [36]:
merged_df.to_csv ("../data/clean_merged_df.csv")
very_large_df.to_csv ("../data/very_large_df.csv")